# README    
1. generate a Keras Layer :AttLayer  
2. Use csv module to import data


# Result  
1)使用Kmer进行输入 没有效果  
    predict 一下 有预测输出，说明训练有效果  
    但是 无论是正负样本，预测成负样本的value远远大于正样本的（0.8》》0.2）  
    可能是训练时正负样本悬殊引起的  
2)正负样本均衡后，acc只有0.61  
    将LSTM的num_unit改到50后，收敛速度加快 但是还是在acc=0.60左右  
3)考虑扩大样本  


In [ ]:
import numpy as np
import pandas as pd
import sys
import os

os.environ['KERAS_BACKEND']='theano'
from keras.engine.topology import Layer
from keras import initializations
from keras import backend as K

from collections  import defaultdict
import pandas as pd
import numpy as np

# Read Data

In [9]:
posdatafile  =  '/home/yinqijin/WorkSpace/2.RNA_Structure_Profile/Gen_data/GM12878_pos_all.fa'
negdatafile  =   '/home/yinqijin/WorkSpace/2.RNA_Structure_Profile/Gen_data/GM12878_neg_all.fa'

poslines = open(posdatafile,'r').readlines()
neglines = open(negdatafile,'r').readlines()

#print poslines[:3]
i=0
posdata =[]
negdata =[]
#print len(poslines)
for item in range(len(poslines)):
    i=i+1
    if i%2== 1:
        continue
    if i%2== 0:
        #print item
        posdata.append(poslines[item])

i=0
for item in range(len(neglines)):
    i=i+1
    if i%2== 1:
        continue
    if i%2== 0:
        negdata.append(neglines[item])
    #print negdata

## Make Promoter/Enhancer Sequence

In [1]:
from pyfasta import Fasta
genome = Fasta('/home/yinqijin/WorkSpace/DataHub/genome.fa')

In [2]:
#获得ｋｍｅｒ的函数
d2 = {'a':0, 'A':0, 'g':1, 'G':1, 'c':2, 'C':2, 't':3, 'T':3, 'N':0, 'n':0}
def seq_to_kspec(seq):
    mat = np.zeros((4096,1))
    k = 0
    if len(seq)<6:
        return mat
    for i in range(6):
        k = k*4 + d2[seq[i]]
    mat[k]+=1  
    for i in range(6,len(seq)):
        k = k - 4**5*d2[seq[i-6]]
        k = k*4 + d2[seq[i]]
        mat[k] += 1  
    return mat

In [ ]:
PESeq=dict()
PESeq['Pro-index']=[]
PESeq['Pro-Seq']=[]
PESeq['Pro-Kmer']=[]
PESeq['Ehr-index']=[]
PESeq['Ehr-Seq']=[]
PESeq['Ehr-Kmer']=[]
PESeq['label'] =[]

In [ ]:
for index in range( len( csvdata[csvkeys[0]])):
    pro_index = [csvdata['promoter_chrom'][index],csvdata['promoter_start'][index],csvdata['promoter_end'][index]]   
    ehr_index = [csvdata['enhancer_chrom'][index],csvdata['enhancer_start'][index] ,csvdata['enhancer_end'][index]]
    pro_seq =  genome[csvdata['promoter_chrom'][index]][int(csvdata['promoter_start'][index]) :int(csvdata['promoter_end'][index])].upper()
    ehr_seq =  genome[csvdata['enhancer_chrom'][index]] [int(csvdata['enhancer_start'][index]) : int(csvdata['enhancer_end'][index])].upper()    

    if len(pro_seq)<6:
        print index,'\tpro_index',pro_index
        continue
    if len(ehr_seq)<6:
        print index,'\tehr_index',ehr_index
        continue
    PESeq['Pro-index'].append(pro_index )
    PESeq['Pro-Seq'].append(pro_seq)
    PESeq['Pro-Kmer'].append( seq_to_kspec(pro_seq))
    PESeq['Ehr-index'].append(ehr_index)
    PESeq['Ehr-Seq'].append(ehr_seq)
    PESeq['Ehr-Kmer'].append(seq_to_kspec(ehr_seq))
    PESeq['label'].append(csvdata['label'][index])

In [ ]:
print '-'*50
for item in PESeq.keys():
    print item, np.shape(PESeq[item])
print 'pos data' , sum(int( item ) for item in PESeq['label'])
print 'neg data',sum(int( 1) if item =='0' else int(0) for item in PESeq['label'])



## cut negative samples

In [ ]:
pos_neg_index = [ int(item) for item in PESeq['label']]
print pos_neg_index[:50]
pos_neg_index = np.array(pos_neg_index )  #很关键

neg_index  = np.where(pos_neg_index==0)
pos_index  = np.where(pos_neg_index ==1)
print neg_index
print pos_index
bal_pos_neg_index = pos_index + neg_index[:len(pos_index)]
print bal_pos_neg_index[:50]
print bal_pos_neg_index[-50:]
print len(bal_pos_neg_index)

In [ ]:
print 'lengthest Enhancer',len(max(PESeq['Ehr-Seq'],key= lambda x:len(x)))
print  'lengthest Promoter',len(max(PESeq['Pro-Seq'],key= lambda x:len(x)))

#Get X,Y with no  limited
X=[]
Y=[]
for index in range( len( PESeq['label'])):
    Y.append(PESeq['label'][index])
    X.append(np.append(PESeq['Pro-Kmer'][index],PESeq['Ehr-Kmer'][index],axis=1))
print np.shape(X)
print np.shape(Y)
X = np.array(X)
Y  = np.array(Y)

index = range(len(PESeq['label']))

In [ ]:
# Get X,Y with   limite  making positive and negative samples 's numbers is the same
X=[]
Y=[]
for index in range( 4220):
    Y.append(PESeq['label'][index])
    X.append(np.append(PESeq['Pro-Kmer'][index],PESeq['Ehr-Kmer'][index],axis=1))
print np.shape(X)
print np.shape(Y)
X = np.array(X)
Y  = np.array(Y)


index = range(4220)

In [ ]:
from keras.utils.np_utils import to_categorical

In [ ]:

VALIDATION_SPLIT =0.2

np.random.seed(1234)

np.random.shuffle(index)
nb_validation_samples = int(VALIDATION_SPLIT*len(Y))

In [ ]:

x_train = X[index[: - nb_validation_samples]]
y_train = Y[index[:-nb_validation_samples]]
x_val = X[index[-nb_validation_samples:]]
y_val =Y[index[-nb_validation_samples:]]



In [ ]:
y_train = to_categorical(y_train) #one-hot 编码labels
y_val = to_categorical(np.asarray(y_val))

In [ ]:
print 'x_train_shape:',np.shape(x_train)
print 'y_train_shape:',np.shape(y_train)

#使验证集正负样本数量相等
print np.shape(y_val)

print 'valid负正样本比例' sum(y_val)


count = defaultdict()
count['0']=[]
count['1']=[]
for i in range(8847):
    if y_val[i][0]==1:
        count['0'].append(i)
    else:
        count['1'].append(i)
print len(count['0']),len(count['1'])


 index_my = count['0'][:401]+count['1']

x_val_my = x_val[index_my,:,:]
y_val_my  =y_val[index_my,:]

print 'x_val_shape:',np.shape(x_val_my)
print 'y_val_shape:',np.shape(y_val_my)

In [ ]:
print y_val_my
print x_val_my

In [ ]:
import hickle as hkl

In [ ]:
hkl.dump(x_val,'./Gen_data/10_Att_BLSTM_x_val.hkl')
hkl.dump(y_val,'./Gen_data/10_Att_BLSTM_y_val.hkl')
hkl.dump(x_train,'./Gen_data/10_Att_BLSTM_x_train.hkl')
hkl.dump(y_train,'./Gen_data/10_Att_BLSTM_y_train.hkl')

In [1]:
import hickle as hkl
x_val = hkl.load('./Gen_data/10_Att_BLSTM_x_val.hkl')
y_val = hkl.load('./Gen_data/10_Att_BLSTM_y_val.hkl')
x_train = hkl.load('./Gen_data/10_Att_BLSTM_x_train.hkl')
y_train = hkl.load('./Gen_data/10_Att_BLSTM_y_train.hkl')

In [2]:
#这一个是负样本均衡过的
import hickle as hkl
x_val = hkl.load('./Gen_data/13_x_val.hkl')
y_val = hkl.load('./Gen_data/13_y_val.hkl')
x_train = hkl.load('./Gen_data/13_x_train.hkl')
y_train = hkl.load('./Gen_data/13_y_train.hkl')

# Make Network

In [3]:
import numpy as np
import pandas as pd
import sys
import os

os.environ['KERAS_BACKEND']='theano'
from keras.engine.topology import Layer
from keras import initializations
from keras import backend as K

from collections  import defaultdict
import pandas as pd
import numpy as np

class AttLayer(Layer):
    def __init__(self, **kwargs):
        self.init = initializations.get('normal')
        #self.input_spec = [InputSpec(ndim=3)]
        super(AttLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape)==3
        #self.W = self.init((input_shape[-1],1))
        self.W = self.init((input_shape[-1],))
        #self.input_spec = [InputSpec(shape=input_shape)]
        self.trainable_weights = [self.W]
        super(AttLayer, self).build(input_shape)  # be sure you call this somewhere!

    def call(self, x, mask=None):


        M = K.tanh(x)
        alpha = K.dot(M,self.W)#.dimshuffle(0,2,1)

        ai = K.exp(alpha)
        weights = ai/K.sum(ai, axis=1).dimshuffle(0,'x')
        weighted_input = x*weights.dimshuffle(0,1,'x')
        return K.tanh(weighted_input.sum(axis=1))
        '''
        eij = K.tanh(K.dot(x, self.W))

        ai = K.exp(eij)
        weights = ai/K.sum(ai, axis=1).dimshuffle(0,'x')

        weighted_input = x*weights.dimshuffle(0,1,'x')
        return weighted_input.sum(axis=1)
        '''
    def get_output_shape_for(self, input_shape):
        return (input_shape[0], input_shape[-1])


Using Theano backend.
Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, cuDNN 5105)


In [4]:
from keras.layers import Embedding
from keras.layers import Dense,Input,Activation
from keras.layers import Embedding, LSTM, Bidirectional,GRU,InputLayer
from keras.models import Model,Sequential
from  keras.regularizers import ActivityRegularizer
from keras.layers.core import Dropout,Flatten

In [5]:
from keras import layers

In [6]:
kmer_input = Input(shape=(4096,2), dtype='float32')

In [9]:
l_lstm =Bidirectional(LSTM(2,return_sequences=True))(kmer_input)

In [10]:
l_lstm_drop = Dropout(0.3)(l_lstm)
l_att = AttLayer()(l_lstm_drop)
l_flatten = Flatten()(l_lstm_drop)
l_att_drop = Dropout(0.5)(l_flatten)
l_dense = Dense(256)(l_att_drop)
l_dense_1 = Dense(64)(l_dense)
l_dense_2 = Dense(32)(l_dense_1)
preds = Dense(len( y_train[0]), activation='softmax',activity_regularizer= ActivityRegularizer(l2=0.005))(l_dense_2)
model  = Model (kmer_input,preds)

In [11]:
model.compile(loss='mse',
              optimizer='rmsprop',
              metrics=['acc'])

print("model fitting - attention LSTM network")
model.summary()


model fitting - attention LSTM network
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 4096, 2)       0                                            
____________________________________________________________________________________________________
bidirectional_2 (Bidirectional)  (None, 4096, 4)       80          input_1[0][0]                    
____________________________________________________________________________________________________
dropout_3 (Dropout)              (None, 4096, 4)       0           bidirectional_2[0][0]            
____________________________________________________________________________________________________
flatten_2 (Flatten)              (None, 16384)         0           dropout_3[0][0]                  
____________________________________________________

In [12]:
model.fit(x_train, y_train, validation_data=(x_val, y_val), nb_epoch=80, batch_size=100)

Train on 3376 samples, validate on 843 samples
Epoch 1/80
3376/3376 [==============================] - 282s - loss: 0.9897 - acc: 0.4935 - val_loss: 0.9729 - val_acc: 0.5125
Epoch 2/80
3376/3376 [==============================] - 276s - loss: 0.9599 - acc: 0.5110 - val_loss: 0.9245 - val_acc: 0.5136
Epoch 3/80
3376/3376 [==============================] - 262s - loss: 0.8971 - acc: 0.5110 - val_loss: 0.8680 - val_acc: 0.4686
Epoch 4/80
3376/3376 [==============================] - 263s - loss: 0.5957 - acc: 0.5560 - val_loss: 0.4961 - val_acc: 0.5765
Epoch 5/80
3376/3376 [==============================] - 257s - loss: 0.5599 - acc: 0.5880 - val_loss: 0.4822 - val_acc: 0.6133
Epoch 6/80
3376/3376 [==============================] - 261s - loss: 0.5432 - acc: 0.6060 - val_loss: 0.4954 - val_acc: 0.5658
Epoch 7/80
3376/3376 [==============================] - 262s - loss: 0.5062 - acc: 0.6386 - val_loss: 0.4786 - val_acc: 0.6335
Epoch 8/80
3376/3376 [==============================] - 259s - l

KeyboardInterrupt: 

In [13]:
model.fit(x_train, y_train, validation_data=(x_val, y_val), nb_epoch=20, batch_size=100)

Train on 3376 samples, validate on 843 samples
Epoch 1/20
 100/3376 [..............................] - ETA: 236s - loss: 0.4181 - acc: 0.8600

KeyboardInterrupt: 

In [ ]:
model.fit(x_train, y_train, validation_data=(x_val, y_val), nb_epoch=20, batch_size=100)

In [ ]:
model.fit(x_train, y_train, validation_data=(x_val, y_val), nb_epoch=50, batch_size=100)

In [18]:
pro = model.predict_on_batch(x_val)

In [19]:
print pro

[[ 0.1240904   0.87590957]
 [ 0.32051146  0.67948854]
 [ 0.72198415  0.27801588]
 ..., 
 [ 0.42740694  0.57259309]
 [ 0.3439495   0.6560505 ]
 [ 0.41883031  0.58116966]]


# Result  
不加Att,3层全连接,0.78


# Save&Search

In [ ]:
dir(model)

In [ ]:
model.__dict__

In [ ]:
model.save_weights('./Gen_data/10_Att-BLSTM.h5')

In [ ]:
model.load_weights('./Gen_data/10_Att-BLSTM.h5',by_name=True)